# Testing VGG-19
A tutorial to test LeNet-5 using ADAPT. Before start tutorial, if you use your GPU, the following cell will set tensorflow to use minimal memory.

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

## Load model to test
The pre-trained VGG-19 is offered by Tensorflow/Keras. You can easily download the pre-trained model using Tensorflow/Keras.

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
model = VGG19()
model.summary()

## Test using ADAPT
From now on, let's test the created model. *The test is done with the Ubuntu VM with 4 cores out of Intel(R) Core(TM) i7-8700K (which have 12 threads in total) and 8GB of RAM.*

### 1. Choose candidate input
ADAPT offers some example images in ```data/imagenet``` folder. We will use ```ILSVRC2012_test_00000242.JPEG```, which is a cute little puppy, for this tutorial.

In [ ]:
from pathlib import Path
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = Path('data/imagenet/ILSVRC2012_test_00000242.JPEG')
img = image.load_img(path, target_size=(224, 224))
img = image.img_to_array(img)
plt.axis('off')
plt.imshow(img.astype(int))

Finally, preprocess the input using provided preprocess function.

In [ ]:
img = preprocess_input(img)

Now our input image is ready!

### 2. Create a fuzzer
ADAPT offers various modules (e.g. coverage metrics and neuron selection strategies) that can be used to compose a fuzzer. First thing to do is wrapping Keras model, since all modules in ADAPT uses a Keras model wrapped with the ```adapt.Network``` class.

In [ ]:
from adapt import Network

In [ ]:
network = Network(model)

Create a neuron coverage with 0.5 as threshold.

In [ ]:
from adapt.metric import NC

In [ ]:
metric = NC(0.5)

Create an adaptive and parameterized neuron selection strategy introduced in the following paper:

    Effective White-box Testing of Deep Neural Networks with Adaptive Neuron-Selection Strategy

In [ ]:
from adapt.strategy import AdaptiveParameterizedStrategy

In [ ]:
strategy = AdaptiveParameterizedStrategy(network)

Now you can compose a fuzzer for the VGG-19 with the chosen input image.

In [ ]:
from adapt.fuzzer import WhiteBoxFuzzer
from tensorflow.keras.applications.vgg19 import decode_predictions

In [ ]:
fuzzer = WhiteBoxFuzzer(network, img, metric, strategy, lr=5, decode=lambda x: decode_predictions(x)[0][0][1])

### 3. Start testing
The given input is tested for 30 minutes, and keep all the inputs generated.

In [ ]:
archive = fuzzer.start(minutes=30, append='all')

### 4. Testing result
You can easily see the summary of the testing result as follows:

In [ ]:
archive.summary()

Here is the coverage graph.

In [ ]:
t, cov = tuple(zip(*archive.timestamp))
plt.plot(t, cov)

Plus, the following is a visualization of some generated images.

In [ ]:
import numpy as np

In [ ]:
fig, ax = plt.subplots(1, len(archive.found_labels), figsize=(len(archive.found_labels) * 2, 2))
for i, label in enumerate(archive.found_labels.keys()):
    ax.set_axis_off()
    ax.title.set_text(str(label))
    im = np.array(archive.inputs[label][0])
    im = np.reshape(im, (224, 224, 3))
    im[:, :, 0] += 103.937
    im[:, :, 1] += 116.779
    im[:, :, 2] += 123.68
    im = im[:, :, ::-1]
    ax.imshow(np.clip(im, 0, 255).astype(int))